# The Preprocessing --Make it Clear and Fine. 
I need to make every data carefully.  
1. Do not filter the spectra with SG-filter, it is not necessary in FPCA.  
2. Do not do any interpolation or re-sample, it is also not necessary in FPCA.  
3. In the FPCA algorithm, be careful in using the bandwidth(In R).  

In [2]:
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.interpolate import interp1d,UnivariateSpline
from scipy.signal import savgol_filter
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
DIR=os.getcwd()
OBDIR=DIR+'/CAT/OB/'
SPDIR=DIR+'/CAT/SP/'
obii=pd.read_csv(OBDIR+'OBII.csv')
spii=pd.read_csv(SPDIR+'SPII.csv')
obiib=pd.read_csv(OBDIR+'OBIIb.csv')
spiib=pd.read_csv(SPDIR+'SPIIb.csv')
obiil=pd.read_csv(OBDIR+'OBIIL.csv')
spiil=pd.read_csv(SPDIR+'SPIIL.csv')
obiip=pd.read_csv(OBDIR+'OBIIP.csv')
spiip=pd.read_csv(SPDIR+'SPIIP.csv')
obiin=pd.read_csv(OBDIR+'OBIIn.csv')
spiin=pd.read_csv(SPDIR+'SPIIn.csv')
sp={'II':spii,'IIb':spiib,'IIL':spiil,'IIP':spiip,'IIn':spiin}
ob={'II':obii,'IIb':obiib,'IIL':obiil,'IIP':obiip,'IIn':obiin}
Shorten=0.001

In [4]:
def strictly_increasing(L):
    return all(x<y for x,y in zip(L,L[1:]))
def fuck_z(wavelength,z):
    return wavelength/(1+z)
def sg(a,window,order,number):
    a[:,1]=savgol_filter(a[:,1],window,order)
    f1=interp1d(a[:,0],a[:,1])
    wavn=np.linspace(a[:,0].min(),a[:,0].max(),num=number)
    anew=np.array([wavn,f1(wavn)])
    return np.transpose(anew)
def spl(a,number):
    f1=interp1d(a[:,0],a[:,1])
    f2=UnivariateSpline(a[:,0],a[:,1])
    wavn=np.linspace(a[:,0].min(),a[:,0].max(),num=number)
    anew=np.array([wavn,f1(wavn)-f2(wavn)])
    return np.transpose(anew)
def walker(specdir):
    i=''
    for i in os.walk(DIR+specdir):
        i
    loc_spec=i[2]
    return loc_spec
def rescaller():
    #Why did I define this function?
    return

In [5]:
def read_spectra(specdir,obloc,percent,shortwave,longwave,log_flux=False):
    loc_spec=walker(specdir)
    obdata=ob[obloc]
    spectra=[]
    chosen_name=[]
    i=0
    for i in loc_spec:
        SNname=i.split('_',1)[1].split('.ascii')[0]
        print(i)
        a=np.genfromtxt(DIR+specdir+i)
        if a.shape[1]==3 or a.shape[1]==7 or a.shape[1]==6:
            a=a[:,0:2]
        assert a.shape[1]==2
        z=float(obdata[obdata['Obj. Name']==SNname]['Redshift'])
        if np.isnan(z):
            continue
        a[:,0]=fuck_z(a[:,0],z)
        if a[:,0].max()<=longwave or a[:,0].min()>=shortwave:
            continue
        if strictly_increasing(a[:,0])==False:continue
        #a=spl(a,5000)
        a=pd.DataFrame({'Wavelength':a[:,0],'Flux':a[:,1]})
        a=a[a['Wavelength']>shortwave]
        a=a[a['Wavelength']<longwave]
        a=np.array(a)
        if a.shape[0]<40:continue
        #a=sg(a,21,1,500)
        a[:,1]=preprocessing.scale(a[:,1])
        spectra.append(a)
        chosen_name.append(i)
        
    spectra_fpca,spectra_rest,name_fpca,name_rest\
    =train_test_split(spectra,chosen_name,test_size=percent)
    
    fpca_flux=np.zeros(0)
    fpca_wave=np.zeros(0)
    fpca_Id=[]
    fpca_category=[]
    for i in range(len(spectra_fpca)):
        fpca_flux=np.concatenate([fpca_flux,spectra_fpca[i][:,1]])
        fpca_wave=np.concatenate([fpca_wave,spectra_fpca[i][:,0]])
        fpca_Id.extend([name_fpca[i].split('_')[0] \
                        for j in range(np.shape(spectra_fpca[i])[0])])
        fpca_category.append(name_fpca[i])
        print(name_fpca[i])
        #if the program fails, sometimes the last line of a file is the troublemaker.
    fpca_data=pd.DataFrame({'ID':fpca_Id,'Flux':fpca_flux,'Wavelength':fpca_wave})
    rest_flux=np.zeros(0)
    rest_wave=np.zeros(0)
    rest_Id=[]
    rest_category=[]
    for i in range(len(spectra_rest)):
        rest_flux=np.concatenate([rest_flux,spectra_rest[i][:,1]])
        rest_wave=np.concatenate([rest_wave,spectra_rest[i][:,0]])
        rest_Id.extend([name_rest[i].split('_')[0] \
                        for j in range(np.shape(spectra_rest[i])[0])])
        rest_category.append(name_rest[i])
        print(name_rest[i])
    rest_data=pd.DataFrame({'ID':rest_Id,\
                            'Flux':rest_flux,\
                            'Wavelength':rest_wave})
    return fpca_data,rest_data,fpca_category,rest_category

In [16]:
def every(where_is_the_spectra=['II','IIL','IIP','IIb','IIn']\
          ,save_the_csv_for_R='Rdata.csv'\
          ,save_the_rest='restdata.csv'\
          ,ratios={'II':0.9,'IIP':0.9,'IIL':0.9,'IIb':0.9,'IIn':0.9}
          ,shortwave=4000,longwave=9000):
    Rdata=[]
    restdata=[]
    Rcat=[]
    restcat=[]
    i=0
    for i in where_is_the_spectra:
        specdir='/'+i+'/'
        fpca_data,rest_data,fpca_category,rest_category=read_spectra(specdir,i,ratios[i],shortwave,longwave)
        Rdata.append(fpca_data)
        restdata.append(rest_data)
        Rcat.extend(fpca_category)
        restcat.extend(rest_category)
    Rdata=pd.concat(Rdata,ignore_index=True)
    Rdata['Wavelength']=(Rdata['Wavelength']-shortwave+Shorten)/(longwave-shortwave+Shorten*2)
    restdata=pd.concat(restdata,ignore_index=True)
    restdata['Wavelength']=(restdata['Wavelength']-shortwave+Shorten)/(longwave-shortwave+Shorten*2)
    Rdata.to_csv(save_the_csv_for_R+'data.csv',encoding='utf-8',index=False)
    restdata.to_csv(save_the_rest+'data.csv',encoding='utf-8',index=False)
    Rcat=pd.DataFrame(Rcat)
    restcat=pd.DataFrame(restcat)
    Rcat.to_csv(save_the_csv_for_R+'cat.csv',encoding='utf-8',index=False)
    restcat.to_csv(save_the_rest+'cat.csv',encoding='utf-8',index=False)
    WaveRescaller={'Shortest Wavelength':[shortwave],'Longest Wavelength':[longwave],'Shorten':[Shorten]}
    WaveRescaller=pd.DataFrame(WaveRescaller)
    WaveRescaller.to_csv('WaveRescaller.csv',encoding='utf-8',index=False)
    return Rcat,restcat

In [17]:
Filter_Elements=['Halpha' ,'Ca','Hbeta','S' ,'FeOMgSi','FeMg','Visible','Expand']
longwaves=      [6900     ,8900,4900   ,5800,5250     ,4600  ,7000     ,9000    ]
shortwaves=     [6150     ,8200,4600   ,5250,4900     ,4200  ,4000     ,4000    ]
for i in range(len(Filter_Elements)):
    Filter_Elements[i]='FPCA-'+Filter_Elements[i]
for i in range(len(Filter_Elements)):
    every(save_the_csv_for_R=Filter_Elements[i]+'/R'\
          #,where_is_the_spectra=['IIP']
          ,save_the_rest=Filter_Elements[i]+'/rest'
          ,shortwave=shortwaves[i],longwave=longwaves[i])

45865_SN2018cwa.ascii
45845_SN2018cur.ascii
45844_SN2018cuf.ascii
45836_SN2018cqi.ascii
45809_SN2018ccb.ascii
45792_SN2018cbb.ascii
45759_SN2018bui.ascii
45760_SN2018bwd.ascii
45737_SN2018bsz.ascii
45731_SN2018bsz.ascii
45716_SN2018bti.ascii
45421_SN2018bei.ascii
45426_SN2018bif.ascii
45387_SN2018bbv.ascii
45402_SN2018bek.ascii
45450_SN2018bge.ascii
45368_SN2018bbv.ascii
45361_SN2018bbr.ascii
45358_SN2018anx.ascii
45286_SN2018awx.ascii
45232_SN2018bl.ascii
45271_SN2018aru.ascii
45273_SN2018aum.ascii
45209_SN2018awe.ascii
45195_SN2018arc.ascii
45194_SN2018arl.ascii
45320_SN2018adu.ascii
45321_SN2018bas.ascii
45121_SN2018afb.ascii
45122_SN2018afb.ascii
45103_SN2018zj.ascii
45091_SN2018aoq.ascii
45080_SN2018ant.ascii
45079_SN2018anh.ascii
45045_SN2018amd.ascii
45073_SN2018amb.ascii
45074_SN2018amc.ascii
45007_SN2018ajz.ascii
44987_SN2017kam.ascii
45075_SN2018aei.ascii
44980_SN2018agn.ascii
45350_SN2018abm.ascii
44977_SN2018afm.ascii
45347_SN2018bbk.ascii
44976_SN2018aad.ascii
45120_SN2018

22442_ASASSN-15ov.ascii
27037_SN2015am.ascii
27041_SN2015af.ascii
27042_SN2015ae.ascii
23261_SN2015S.ascii
23262_SN2015S.ascii
27056_PSNJ17292918+7542390.ascii
27058_ASASSN-15ln.ascii
39367_iPTF14hls.ascii
27063_PSNJ14372160+3634018.ascii
39366_iPTF14hls.ascii
39365_iPTF14hls.ascii
22198_iPTF14hls.ascii
45356_SN2015cq.ascii
39364_iPTF14hls.ascii
25033_iPTF14hls.ascii
42936_SN1992as.ascii
21960_PS15aaa.ascii
21980_iPTF15agp.ascii
27064_SN2015R.ascii
21959_PS15zf.ascii
39362_iPTF14hls.ascii
25032_iPTF14hls.ascii
43587_OGLE-2014-SN-073.ascii
27066_ASASSN-15fz.ascii
21922_iPTF15vo.ascii
21859_LSQ15xp.ascii
39361_iPTF14hls.ascii
27071_SN2015X.ascii
25031_iPTF14hls.ascii
21775_LSQ15ok.ascii
27072_PSNJ14002117+3854517.ascii
21742_LSQ15mc.ascii
39360_iPTF14hls.ascii
21599_LSQ15kp.ascii
21589_CSS150210-085514-080940.ascii
21727_iPTF15jg.ascii
21572_OGLE-2015-SN-043.ascii
39359_iPTF14hls.ascii
43586_OGLE-2014-SN-073.ascii
43888_iPTF14hls.ascii
27081_PSNJ11105565+5322493.ascii
21529_OGLE-2015-SN-

27198_SN2012gg.ascii
27320_SN2012gf.ascii
27789_PTF12kso.ascii
27321_SN2012gc.ascii
10875_SNHunt148.ascii
19852_SNHunt148.ascii
10874_SN2012ft.ascii
19484_SN2012ft.ascii
10860_LSQ12fhh.ascii
19192_LSQ12fhh.ascii
10831_SN2012ew.ascii
19455_SN2012ew.ascii
19492_PTF12iqw.ascii
10802_LSQ12eor.ascii
19175_LSQ12eor.ascii
10590_LSQ12egy.ascii
19163_LSQ12egy.ascii
10586_LSQ12egy.ascii
19162_LSQ12egy.ascii
27788_PTF12gcx.ascii
27262_SN2012ed.ascii
27787_PTF12gcx.ascii
37913_SN2002hh.ascii
27786_PTF12gcx.ascii
10698_PTF12gnn.ascii
8792_MASTERJ134201.21-023856.2.ascii
19389_MASTERJ134201.21-023856.2.ascii
8776_CSS120421-143501-071918.ascii
8783_LSQ12cer.ascii
8786_Sn2012ga.ascii
19008_CSS120421-143501-071918.ascii
19104_LSQ12cer.ascii
19485_Sn2012ga.ascii
27276_SN2012by.ascii
27277_SN2012bv.ascii
8716_SNHunt119.ascii
19853_SNHunt119.ascii
8706_LSQ12bri.ascii
19067_LSQ12bri.ascii
27283_SN2012bn.ascii
27293_SN2012ak.ascii
27212_SN2012ai.ascii
27218_SN2012W.ascii
27217_SN2012S.ascii
27220_SN2012K.as

KeyboardInterrupt: 

0